In [1]:
import pandas as pd
import random
import warnings
warnings.filterwarnings('ignore')
from prod_details import Parameters
pm = Parameters()


In [2]:
# Load the data, then define the index of some data to simplify future data indexing.

outbound_data = pd.read_excel("Outbound.xlsx")
outbound_data.index = outbound_data.State
outbound_data = outbound_data.drop(['State', 'Small shipment'], axis=1)

demand_data = pd.read_excel('Demand Forecast.xlsx')
demand_data.index = demand_data.state

unit_data = pd.read_excel('Product Data per State.xlsx')
oc_data = pd.read_excel('Opening-Closing Costs.xlsx')
unit_data.index = unit_data.state
oc_data.index = oc_data.State

costs_data = pd.read_excel('Opening-Closing Costs.xlsx')
costs_data.index = costs_data.State

product_data = pd.read_excel('Product Master Data.xlsx')
product_data.index = product_data.COMM_NAME

warehousing_handlingOut_data = pd.read_excel('Warehousing.xlsx', sheet_name='Handling Out')
warehousing_handlingIn_data = pd.read_excel('Warehousing.xlsx', sheet_name='Handling In')
warehousing_storageCost = pd.read_excel('Warehousing.xlsx', sheet_name='Storage')

warehousing_handlingOut_data.index = warehousing_handlingOut_data.DC
warehousing_handlingIn_data.index = warehousing_handlingIn_data.DC
warehousing_storageCost.index = warehousing_storageCost.DC


In [3]:
# As-Is Situation

# Define the as-is DC allocation. The keys are the DC, the values are the states where the corresponding DC's customers are.
as_is_dc = {
    "NY": ['CT', 'DC', 'DE', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT'],
    "ND": ['MN', 'MT', 'ND', 'SD', 'WY'],
    "IL": ['HI', 'IA', 'IL', 'IN', 'KS', 'KY', 'MI', 'MO', 'NE', 'OH', 'VA', 'WI', 'WV'],
    "TN": ['AL', 'FL', 'GA', 'NC', 'SC', 'TN'],
    "WA": ['AK', 'ID', 'OR', 'WA'],
    "TX": ['AR', 'AZ', 'LA', 'MS', 'NM', 'OK', 'TX'],
    "UT": ['CA', 'CO', 'NV', 'UT']
}


all_costs = pm.total_costs(as_is_dc)
print('Total Costs:', round(all_costs[0]))
print('Total Inbound Costs:', round(all_costs[1]))
print('Total Warehousing Costs:', round(all_costs[2]))
print('Total Outbound Costs:', round(all_costs[3]))
print('Total Operational Costs:', round(all_costs[4]))
# print(pm.handling_out_costs(as_is_dc, pm.unit_data, pm.warehousing_handlingOut_data))
# dc_product_demand = pm.product_demand_per_dc(as_is_dc, pm.demand_data, pm.product_data)
# dc_product_demand_container = pm.product_demand_per_dc(as_is_dc, pm.demand_data, pm.product_data, False)
# vol = pm.volume_per_dc(dc_product_demand)
# st = pm.storage_costs(vol, pm.warehousing_storageCost)
# print(st)
# print(pm.handling_in_costs(dc_product_demand_container, pm.warehousing_handlingIn_data))

Total Costs: 10224747
Total Inbound Costs: 1124750
Total Warehousing Costs: 2083496
Total Outbound Costs: 1141251
Total Operational Costs: 7000000


In [4]:
# %run warehousingCosts.ipynb
import import_ipynb
from warehousingCosts import Warehousing
wh = Warehousing(as_is_dc)

importing Jupyter notebook from warehousingCosts.ipynb


In [14]:
pm.total_costs({'WA': ['AK', 'ID', 'OR', 'WA'], 'ND': ['IL', 'MI'], 'IL': ['LA', 'NM', 'OK', 'TX'], 'TX': ['IA', 'IN', 'MN', 'MT', 'ND', 'NE', 'SD', 'WI'], 'TN': ['AL', 'AR', 'FL', 'GA', 'KS', 'KY', 'MO', 'MS', 'NC', 'OH', 'PA', 'SC', 'TN', 'VA', 'WV'], 'UT': ['AZ', 'CA', 'CO', 'NV', 'UT', 'WY'], 'NY': ['CT', 'DE', 'HI', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT', 'DC']}
)

[10363761.617744341, 1124750, 1903790.55, 1459971.06774434, 7000000]

In [6]:
# Algorithm 1: Greedy Constructive Search
# Allocate dc based on the distance/costs greedily 
greedy_dc_allocation = {}
outbound_data = pm.outbound_data
for state in outbound_data.index:
    best_dc = (outbound_data.loc[state]).idxmin()
    
    if best_dc not in greedy_dc_allocation:
        greedy_dc_allocation[best_dc] = [state]
    elif isinstance(greedy_dc_allocation[best_dc], list):
        greedy_dc_allocation[best_dc].append(state)
    else:
        greedy_dc_allocation[best_dc] = [greedy_dc_allocation[best_dc], state]

all_costs = pm.total_costs(greedy_dc_allocation, as_is_dc)
print('Total Costs:', round(all_costs[0]))
print('Total Inbound Costs:', round(all_costs[1]))
print('Total Warehousing Costs:', round(all_costs[2]))
print('Total Outbound Costs:', round(all_costs[3]))
print('Total Operational Costs:', round(all_costs[4]))

print(greedy_dc_allocation)

# Turns out, even though the shipping costs are lower,
# the total outbound costs are relatively higher since the operating and opening costs are skyrocketed.

Total Costs: 12955815
Total Inbound Costs: 1124750
Total Warehousing Costs: 2036492
Total Outbound Costs: 919322
Total Operational Costs: 10000000
{'WA': ['AK', 'ID', 'MT', 'OR', 'WA'], 'TN': ['AL', 'FL', 'GA', 'KY', 'MS', 'NC', 'OH', 'SC', 'VA', 'WV'], 'TX': ['AR', 'LA', 'NM', 'OK', 'TN', 'TX'], 'UT': ['AZ', 'CO', 'NV', 'UT', 'WY'], 'CA': ['CA'], 'NY': ['CT', 'DE', 'MA', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT'], 'PA': ['HI', 'MD', 'PA', 'DC'], 'KS': ['IA', 'KS', 'MO', 'NE'], 'IL': ['IL', 'MI', 'MN'], 'ND': ['IN', 'ND', 'SD', 'WI']}


In [15]:
# Algorithm 1b: Greedy Constructive Heuristics
# Now, knowing that opening/closing and operating new DC can be costly, we apply the same algorithm without opening a new DC.

greedy_dc_allocation = {}
outbound_data = pm.outbound_data

for state in outbound_data.index:
    current_dc = outbound_data[list(as_is_dc.keys())]
    best_dc = (current_dc.loc[state]).idxmin()
    if best_dc not in as_is_dc:
        print(best_dc)
    
    if best_dc not in greedy_dc_allocation:
        greedy_dc_allocation[best_dc] = [state]
    elif isinstance(greedy_dc_allocation[best_dc], list):
        greedy_dc_allocation[best_dc].append(state)
    else:
        greedy_dc_allocation[best_dc] = [greedy_dc_allocation[best_dc], state]

all_costs = pm.total_costs(greedy_dc_allocation, as_is_dc)
print('Total Costs:', round(all_costs[0]))
print('Total Inbound Costs:', round(all_costs[1]))
print('Total Warehousing Costs:', round(all_costs[2]))
print('Total Outbound Costs:', round(all_costs[3]))
print('Total Operational Costs:', round(all_costs[4]))
print(greedy_dc_allocation)
# The shipping costs are lowered again, but the handling costs are increasing slightly.
# It resulted in a less significant decrease.

Total Costs: 9817339.448692791
Total Inbound Costs: 1124750
Total Warehousing Costs: 1874181
Total Outbound Costs: 943159
Total Operational Costs: 7000000
{'WA': ['AK', 'ID', 'OR', 'WA'], 'TN': ['AL', 'AR', 'FL', 'GA', 'KS', 'KY', 'MO', 'MS', 'NC', 'OH', 'PA', 'SC', 'TN', 'VA', 'WV'], 'UT': ['AZ', 'CA', 'CO', 'NV', 'UT', 'WY'], 'NY': ['CT', 'DE', 'HI', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT', 'DC'], 'ND': ['IA', 'IN', 'MN', 'MT', 'ND', 'NE', 'SD', 'WI'], 'IL': ['IL', 'MI'], 'TX': ['LA', 'NM', 'OK', 'TX']}


In [8]:
# Best solution
 

to_be = {'WA': ['AK', 'ID', 'OR', 'WA'], 
         'TN': ['AL', 'AR', 'FL', 'GA', 'KS', 'KY', 'MO', 'MS', 'NC', 'OH', 'PA', 'SC', 'TN', 'VA', 'WV'],
         'UT': ['AZ', 'CA', 'CO', 'NV', 'UT', 'WY'], 
         'NY': ['CT', 'DE', 'HI', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT', 'DC'], 'ND': ['IA', 'IN', 'MN', 'MT', 'ND', 'NE', 'SD', 'WI'], 'IL': ['IL', 'MI'], 'TX': ['LA', 'NM', 'OK', 'TX']}

# to_be = {'WA': ['AK', 'ID', 'OR', 'WA'], 
#          'TN': ['AL', 'AR', 'FL', 'GA', 'KS', 'KY', 'MO', 'MS', 'NC', 'OH', 'PA', 'SC', 'TN', 'VA', 'WV'],
#          'UT': ['AZ', 'CA', 'CO', 'NV', 'UT', 'WY'], 
#          'NY': ['CT', 'DE', 'HI', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT', 'DC'], 'ND': ['IA', 'IN', 'MN', 'MT', 'ND', 'NE', 'SD', 'WI'], 'IL': ['IL', 'MI'], 'TX': ['LA', 'NM', 'OK', 'TX']}
all_costs = pm.total_costs(to_be, as_is_dc)
print('Total Costs:', round(all_costs[0]))
print('Total Inbound Costs:', round(all_costs[1]))
print('Total Warehousing Costs:', round(all_costs[2]))
print('Total Outbound Costs:', round(all_costs[3]))
print('Total Operational Costs:', round(all_costs[4]))
print(greedy_dc_allocation)

Total Costs: 9817339
Total Inbound Costs: 1124750
Total Warehousing Costs: 1874181
Total Outbound Costs: 943159
Total Operational Costs: 7000000
{'WA': ['AK', 'ID', 'OR', 'WA'], 'TN': ['AL', 'AR', 'FL', 'GA', 'KS', 'KY', 'MO', 'MS', 'NC', 'OH', 'PA', 'SC', 'TN', 'VA', 'WV'], 'UT': ['AZ', 'CA', 'CO', 'NV', 'UT', 'WY'], 'NY': ['CT', 'DE', 'HI', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'RI', 'VT', 'DC'], 'ND': ['IA', 'IN', 'MN', 'MT', 'ND', 'NE', 'SD', 'WI'], 'IL': ['IL', 'MI'], 'TX': ['LA', 'NM', 'OK', 'TX']}
